In [ ]:
!pip install pinecone-client transformers flask requests

In [ ]:
!pip install sentence-transformers

In [ ]:
!pip install odfpy

In [55]:
!mkdir -p docs_odt  # Si no existe la carpeta lo creamos
!mv isaias_cv.odt  octavio_cv.odt santiago_cv.odt docs_odt/

In [4]:
import pinecone
import re
import os
import odf.opendocument
import odf.text
import odf.element
from sentence_transformers import SentenceTransformer
from transformers import pipeline
from pinecone import Pinecone
from pinecone import ServerlessSpec
from sentence_transformers import SentenceTransformer

In [5]:
PINECONE = {
    "API_KEY": "pcsk_2b78HU_TqDQRdqmBRhgvc6RFtLamaqMW6NTqq7kziXPLcZEa4aabk5iTSwraqVCx3dF6iy",
    "ENVIRONMENT": "us-west1-gcp"
}

In [6]:
# Initialize Pinecone by creating an instance of the Pinecone class
pinecone = Pinecone(api_key=PINECONE["API_KEY"], environment=PINECONE["ENVIRONMENT"]) # Changed this line

index_name = "rag-index"

In [7]:
index = pinecone.Index(index_name)
# Modelo para embeddings y LLM
embedder = SentenceTransformer('all-MiniLM-L6-v2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [60]:
# Función para leer y extraer texto desde archivos .odt
def leer_documentos_odt(ruta_archivo):
    documento = odf.opendocument.load(ruta_archivo)
    contenido = []
    for elemento in documento.getElementsByType(odf.text.P):
        # Extraer el texto de cada elemento <P>
        texto_parrafo = ''.join([node.data for node in elemento.childNodes if isinstance(node, odf.element.Text)])
        if texto_parrafo.strip():  # Asegurarse de que el texto no esté vacío
            contenido.append(texto_parrafo.strip())
    return "\n".join(contenido)

# Directorio que contiene los archivos ODT
ruta_directorio = "/content/docs_odt"  # Cambia esta ruta según tu caso

# Subir los documentos al índice de Pinecone
for nombre_archivo in os.listdir(ruta_directorio):
    if nombre_archivo.endswith(".odt"):
        ruta_archivo = os.path.join(ruta_directorio, nombre_archivo)
        try:
            # Leer el contenido del archivo
            contenido_documento = leer_documentos_odt(ruta_archivo)

            # Generar el embedding
            vector = embedder.encode(contenido_documento).tolist()

            # Definir un namespace basado en el nombre del archivo (sin extensión)
            namespace = os.path.splitext(nombre_archivo)[0].lower()

            # Subir el vector al índice con metadatos
            metadata = {"text": contenido_documento, "filename": nombre_archivo}
            index.upsert([(namespace, vector, metadata)], namespace=namespace)
            print(f"Archivo {nombre_archivo} subido correctamente al namespace '{namespace}'.")
        except Exception as e:
            print(f"Error al procesar {nombre_archivo}: {e}")

Archivo isaias_cv.odt subido correctamente al namespace 'isaias_cv'.
Archivo santiago_cv.odt subido correctamente al namespace 'santiago_cv'.
Archivo octavio_cv.odt subido correctamente al namespace 'octavio_cv'.


In [8]:
# Verificar el estado del índice
stats = index.describe_index_stats()
print(stats)

{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {'isaias_cv': {'vector_count': 1},
                'octavio_cv': {'vector_count': 1},
                'santiago_cv': {'vector_count': 1}},
 'total_vector_count': 3}


# El índice ya existe en Pinecone , dado que fue creado para el trabajo anterior.

In [32]:
llm = pipeline("text-generation", model="gpt2", max_length=100)
#llm = pipeline("text2text-generation", model="google/flan-t5-base")

In [33]:
# Decidir el agente basado en el prompt
def decide_agent(prompt):
    if re.search(r'isaias', prompt, re.IGNORECASE):
        return "isaias_cv"
    elif re.search(r'octavio', prompt, re.IGNORECASE):
        return "octavio_cv"
    elif re.search(r'santiago', prompt, re.IGNORECASE):
        return "santiago_cv"
    else:
        return None

In [47]:
def retrieve_context(prompt, namespace):
    # Connect to the Pinecone index
    index = pinecone.Index(index_name)
    vector = embedder.encode(prompt).tolist()
    # Change this line to use keyword arguments for 'vector'
    result = index.query(vector=vector, top_k=1, namespace=namespace)
    if result["matches"]:
        # Access metadata directly as a dictionary
        return result["matches"][0].metadata['text']
    else:
        # Return a message indicating no context was found to avoid the error
        return "No se encontró información relevante para la consulta."

In [51]:
# Generar respuesta usando el LLM
#def generate_response(prompt, context):
#    full_prompt = f"Contexto: {context}\nPregunta: {prompt}\nRespuesta:"
#    response = llm(full_prompt)
#    return response[0]['generated_text']
def generate_response(prompt, context):
    full_prompt = f"Contexto: {context}\nPregunta: {prompt}\nRespuesta:"
    response = llm(full_prompt, max_new_tokens=50)[0]  # Get the first element
    return response['generated_text']  # Access the generated text
# Sistema principal
def multiagent_system(prompt):
    agent = decide_agent(prompt)
    if agent:
        context = retrieve_context(prompt, namespace=agent)
        return generate_response(prompt, context)
    else:
        return "No se pudo determinar el agente para la consulta."

In [52]:
# Consulta ejemplo al namespace 'isaias'
query_text = "¿Qué experiencia tiene Isaias?"
query_vector = embedder.encode(query_text).tolist()

result = index.query(vector=query_vector, top_k=1, namespace="isaias_cv")
print(result)

{'matches': [{'id': 'isaias_cv', 'score': 0.165477365, 'values': []}],
 'namespace': 'isaias_cv',
 'usage': {'read_units': 5}}


In [ ]:
while True:
  prompt = input("Ingresa tu consulta (o escribe 'salir' para terminar): ")
  if prompt.lower() == "salir":
    break
  response = multiagent_system(prompt)
  print(response)